# Detekce defektů pomocí Fourierovy transformace
Na dnešním cvičení bude úkolem vytvořit algoritmus na kontrolu kvality kartonu (tzv. vlnité lepenky). Algoritmus by měl být schopný **detekovat trhliny, díry a jiné defekty**. Jelikož kontrolovaný karton je vyráběn jako tzv. nekonečný pás, je potřeba využít řádkovou kameru. 

<img src="images/vlnita_lepenka.jpg" width="50%">

Výhodou materiálu je, že má **opakující se** strukturu. Tím pádem lze k detekci vytvořit algoritmus využívající **Fourierovu transformaci** (rozklad obrazu na opakující se obrazové frekvence). Na našem konkrétním kartonu se defekty vyskytují v podobě promáčklin na některých vlnách. Tím byl **narušen** jejich **pravidelný vzor** a tato nedokonalost je patrná ve frekvenčním spektru obrazu. Úkolem je tyto defekty v obrázku **najít** za pomoci Fourierovy transformace a **vyznačit**.

Obrázek kartonu byl nasnímán řádkovou kamerou. Kamera byla umístěna staticky nad pásem, karton se pod ní pohyboval pomocí důmyslného dopravníku.


## Řádkové kamery
Řádková kamera neboli řádkový skener je speciální druh kamery, která snímá pouze 1 řádek. Díky tomu je schopna dosáhnout obrovských frekvencí snímání (až ticíce řádků za 1 sekundu). V labu máme kamery [Basler Racer](https://www.baslerweb.com/en/products/cameras/line-scan-cameras/racer/) o velikosti řádku 6k, 8k a 12k pixelů. Řádkové kamery jsou díky svým specifickým vlastnostem vynikající pro řadu úloh, typickým použitím je snímání nekonečného pásu ve výrobních linkách.

<img src="images/racer.png" width="25%">

Z podstaty skeneru však vyplývá, že se objekt pod kamerou musí pohybovat. V případě, že se objekt nebude pohybovat, vytvoří kamera obrázek, který bude mít všechny řádky totožné (pouze ten 1, který kamera snímá).

Tím, že je kamera schopná dosahovat neskutečných frekvencí snímání dále umožňuje získávat obrazová data v neskutečně velkém rozlišení. Na druhou stranu nás tím nutí nastavovat nízkou dobu expozice a tedy je zapotřebí mnohem více světla.

### Import knihoven a konfigurace

In [ ]:
import os
import io

import cv2
import numpy as np
import matplotlib.pyplot as plt
import pytesseract

from improutils import *

%matplotlib inline
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

### Pomocné funkce

Z funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

- [`load_image(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/acquisition/img_io.py#L11)
- [`plot_images(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/visualisation/visualisation.py#L11)


- [`crop(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/preprocessing/preprocessing.py#L94)
- [`normalize(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/preprocessing/preprocessing.py#L65)
- [`filtration_median(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/filtration/filtration.py#L21)
- [`segmentation_one_threshold(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/segmentation/segmentation.py#L50)
- [`segmentation_auto_threshold(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/segmentation/segmentation.py#L67)
- [`segmentation_two_thresholds(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/segmentation/segmentation.py#L82)
- [`segmentation_adaptive_threshold(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/segmentation/segmentation.py#L100)

Následující funkce jsou nové a umožňují nám provést Fourierovu transformaci a související operace.

- [`apply_fft(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/filtration/filtration.py#L53)
- [`create_filter_mask(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/filtration/filtration.py#L95)
- [`filter_mag_spec(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/filtration/filtration.py#L117)
- [`inverse_fft(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/filtration/filtration.py#L73)

---

## Úkol

### 1) Načtěte a zobrazte vstupní data nasnímaného kartonu.

Pozor, obrázek je potřeba načíst jako černobílý, tzn. pouze s jedním kanálem. Lze toho docílit vícero způsoby: buď lze použít klasickou funkci `load_image` a vybrat libovolný kanál nebo obrázek následně převést na šedotónový.

In [ ]:
cardboard_texture = load_image(...) ### načtení dat
cardboard_texture = ...(cardboard_texture) ### převod do šedotónu
plot_images(cardboard_texture)

### 2) Vyberte si z obrazu rozumně velký čtvercový výřez s nějakou vadou na kartonu.
Velikost obrazu zvolte 500 × 500.

In [ ]:
cardboard_texture_cropped = crop(cardboard_texture, ..., ..., ..., ...) ### zvolte správně ořez

# Kontrola rozměrů obrazu
h,w = cardboard_texture_cropped.shape
if h != w:
    print('Obrázek není čtverec! [h × w] = ' + str(h) + ' × ' + str(w) + '!')

plot_images(cardboard_texture_cropped)

### 3) Vyfiltrujte v rozumné míře šum mediánovým filtrem vhodné velikosti.
_Pro připomenutí_, mediánový filtr zachovává hrany v obraze (tzn. nechová se jako filtr dolní propusti). Proto je vhodné jeho využití společně s Fourierovou transformací. V případě využití filtrů dolní propusti (blur, gauss) by byly vyšší obrazové frekvence zbytečně odstraněny.

In [ ]:
cardboard_texture_filtered = ...(cardboard_texture_cropped, ...) ### filtrace o vhodné velikosti
plot_images(cardboard_texture_filtered)

### 4) Zvyšte konstrast obrazu použitím min-max normalizace.
Normalizace obrazu převede jeho aktuální rozsah hodnot na zvolené hodnoty. V případě zvyšování konstrastu jsou zvolenými hodnotami minimální resp. maximální hodnoty pro osmibitový obraz (tedy 0 - 255). 

In [ ]:
cardboard_normalized = ...(cardboard_texture_filtered) ###
plot_images(cardboard_normalized)

### 5)  Zobrazte si frekvenční spektrum obrázku.
Aplikujte Fourierovu transformaci na upravený obrázek a získané spektrum si zobrazte. 

_Poznámka:_ Frekvenční spektrum obrazu je tvořené komplexními čísly (viz [Předzpracování obrazu - Filtrace v prostorové a frekvenční oblasti](https://courses.fit.cvut.cz/BI-SVZ/lectures/files/bi-svz-07-filtrace-v-prostorove-a-frekvencni-oblasti.pdf). Pro jeho správné funkční zobrazení se využívá následujících pár vylepšení:
- Zobrazení velikosti těchto komplexních čísel.
- Zobrazení obrázku spektra v logaritmickém měřítku (z důvodu neporovnatelně velké velikosti stejnosměrné složky = nulové frekvence oproti zbylým frekvencím). 
- Zobrazení spektra středově souměrného než je vypočtené pomocí přeházení kvadrantů (tzv. _shift_).


In [ ]:
mag_spec, spectrum_shift = ...(cardboard_normalized) ### aplikujte Fourierovu transformaci
plot_images(cardboard_normalized, mag_spec)

### 6) Nalezněte souřadnice zajímavých frekvencí ve frekvenčním spektru.
S vizualizací frekvenčního spektra lze zacházet jako s klasickým obrázkem. Jedná se přeci jen o osmibitový obraz s hodnotami 0 - 255. 

Segmentujte obraz tak, aby v něm zbylo pouze **7 pixelů** (neboli hodnot zajímavých obrazových frekvencí). Hodnotu **amplitudy** obrazové frekvence rezprezentuje **jas** v daném pixelu. Nejvyšší hodnota jasu v obrazu je samozřejmě jeho stejnosměrná složka. 

In [ ]:
# Segmentace
threshold = ... ### zvolte správný práh
spectrum_intresting = ...(mag_spec, threshold) ### zvolte správnou metodu segmentace

# Nalezení indexů
idx = cv2.findNonZero(spectrum_intresting)
if len(idx) != 7:
    print('Pozor nalezených frekvencí není 7!')

# Výpis hodnot
print(idx)

### 7) Experimentujte s frekvenčními filtry. 
Vytvořte několik filtrů **(4)** pro představu toho, co filtrace spektra dělá. Vždy si zobrazte kombinaci **filtr + filtrované spektrum + obrázek s vyfiltrovaným spektrem**. Filtrace obrazových frekvencí se provádí nad získaným spektrem. Ve spektru ukazují vysoké hodnoty na zajímavé frekvence vyskytující se v obraze. 

Vytvořte 4 takovéto filtry:
1. Filtr pouze stejnosměrné složky obrazu.
2. Filtr dolní propust v ose x i ose y.
3. Filtr dolní propust pouze v ose y obsahující zajímavé hodnoty frekvenčního spektra kartonu.
4. Selektivní filtr v ose y pouze 7 (6+1) zajímavých hodnot frekvencí ze spektra kartonu. 

Vhodné filtry mohou vypadat následovně:
![](images/filters.png)

Hint k práci s funkcí `create_filter_mask()`:
- Vstupem je **vždy list** hodnot, např. `[1,2,3,4,5,6,7]`.
- Pokud chcete jako vstup použít `range()`, je třeba ho zapsat jako `*range(1:7)`.
- Pokud chcete kombinovat obojí, lze to např. takto: `[1, *range(3-5), 7]`.
- V listu nemusí být pouze unikátní čísla (nespadne to), ale není jediný důvod, proč by tam mělo být nějaké číslo víckrát :)

In [ ]:
# Filtr 1 pouze stejnosměrná složka
filter_mask_1 = ...(cardboard_normalized.shape, [...], [...]) ### doplňte jak funkci, tak hodnoty
spec_mask_filt_1 = ...(mag_spec, filter_mask_1) ### doplňte funkci
cardboard_filt_1 = ...(spectrum_shift, filter_mask_1) ### doplňte funkci

# Filtr 2 dolní propust v x i y
filter_mask_2 = ...(cardboard_normalized.shape, [...], [...]) ### doplňte jak funkci, tak hodnoty
spec_mask_filt_2 = ...(mag_spec, filter_mask_2) ### doplňte funkci
cardboard_filt_2 = ...(spectrum_shift, filter_mask_2) ### doplňte funkci

# Filtr 3 dolní propust v y se zajímavými hodnotami
filter_mask_3 = ...(cardboard_normalized.shape, [...], [...]) ### doplňte jak funkci, tak hodnoty
spec_mask_filt_3 = ...(mag_spec, filter_mask_3) ### doplňte jak funkci, tak hodnoty
cardboard_filt_3 = ...(spectrum_shift, filter_mask_3) ### doplňte funkci

# Filtr 4 selektivní pouze pro 7 zajímavých hodnot v obraze
filter_mask_4 = ...(cardboard_normalized.shape, [...], [...]) ### doplňte jak funkci, tak hodnoty
spec_mask_filt_4 = ...(mag_spec, filter_mask_4) ### doplňte funkci
cardboard_filt_4 = ...(spectrum_shift, filter_mask_4) ### doplňte funkci

# Zobrazení
plot_images(
    filter_mask_1, spec_mask_filt_1, cardboard_filt_1,
    filter_mask_2, spec_mask_filt_2, cardboard_filt_2,
    filter_mask_3, spec_mask_filt_3, cardboard_filt_3,
    filter_mask_4, spec_mask_filt_4, cardboard_filt_4,
    normalize=True
)

### 8) Zjistěte index správné obrazové frekvence (ind), která odpovídá struktuře (vzoru) materiálu kartonu.
Je potřeba se zamyslet nad strukturou materiálu. 
- Jeho "vlnitost" neboli opakování vzoru je pouze v ose y obrazu, v ose x jsou hodnoty vždy stejné. 
- Jaká je perioda opakování obrazového vzoru (T_vzor)?
- Jaká je maximální vzorkovací frekvence, kterou obraz umožňuje pojmout (f_max)?
- Zajímavých frekvencí je 7, ale jen 1 odpovídá přesně vzoru.
- Stejnosměrná složka to NENÍ, i když se to tak může zdát.

Vysoké frekvence (zde odpovídají vadám a šumu) a nízké frekvence odpovídají struktuře materiálu. To jsou ty, které jsou blíže ke středu obrázku spektra (našich 7 vybraných).

Vhodné vzorce:
$$ f_{vzor} = \frac{1}{T_{vzor}} \; \mathrm{[pix}^{-1}] $$

$$ f_{max} = \frac{1}{T_{max}} \; \mathrm{[pix}^{-1}] $$

$$ pix_{spect} = f_{vzor} \cdot T_{max} \; \mathrm{[pix]} $$

Hodnota `pix_spect` označuje hledanou vzdálenost v pixelech bodu obrazové frekvence od stejnosměrné složky spektra (středu) v obrazovém frekvenčím spektru.

In [ ]:
# Změřte si periodu vzoru
# ... uložte si obrázek na disk 
# ... otevřete si ho v nějakém editoru, ve kterém jste schopni určit souřadnice dvou pixelů
save_image(cardboard_normalized, 'data/cardboard_normalized.png')

In [ ]:
# Podpůrné indexy a hodnoty
indexes = idx[:, 0, 1]
idx_middle = math.floor(len(indexes)/2)

# Hodnota pixelu indexu stejnosměrné složky
ind_ss = indexes[idx_middle]
indexes = np.delete(indexes, idx_middle)

# Maximální perioda v obraze T_max (rozlišení obrazu) [pix]
T_max = ... ### získaná hodnota

# Perioda opakování struktury (vzoru) [pix}
T_vzor = ... ### změřená hodnota

# Frekvence vzoru [pix^-1]
f_vzor = ... ### vzorec

# Hledaná hodnota frekvence se pozná podle vzdálenosti od středu spektra [pix]
pix_spect = ... ### vzorec

# Hledání ideální frekvence ze zjištěných možných
diffs = []
for i in indexes:
    pix_from_center = abs(i - ind_ss)
    
    # Nemá smysl odečítat střed sám od sebe
    if pix_from_center != 0:
        diff = abs(pix_spect - pix_from_center)
        diffs.append(diff)

# Převod na numpy array kvůli vyhledání více možností než 1
diffs = np.array(diffs)
    
# Nalezení správných indexů obrazové frekvence
ind = indexes[np.where(diffs == diffs.min())]
    
# Výpis výsledků
print('Správné obrazové frekvence leží na hodnotách pixelů na ose y: ' + str(ind))
print('Spektrum je totiž souměrné podle středu.')

### 9) Vytvořte finální filtr a odstraňte z obrazu opakující se vzor.
Opět vytvořte filtr pro frekvenční spektrum, který tentokrát ideálně odstraní z obrazu **POUZE** opakující se vzor. Filtr použijte na původní frekvenční spektrum a aplikujte **inverzní fourierovu transformaci**.

_Pozor_: v předchozích případech se takový vzor hledal, teď je nutné ho odstranit!

In [ ]:
# Vytvořte vzor pro odstranění opakujícího se vzoru
filter_mask_final = ...(cardboard_normalized.shape, [...], [...]) ### doplňte jak funkci, tak hodnoty

# Funguje filtr podle zadání?

# Vytvořte filtrované spektrum a výsledný filtrovaný obraz
spec_mask_filt_final = ...(mag_spec, filter_mask_final) ### doplňte funkci
cardboard_filt_final = ...(spectrum_shift, filter_mask_final).astype(np.uint8) ### doplňte funkci

plot_images(spec_mask_filt_final, cardboard_filt_final, normalize=True)

### 10) Detekujte defekty materiálu. Vhodně vizualizujte.
Segmentujte defekty pomocí vhodného prahování. Cílem je najít pouze největší defekty. Drobnosti lze ignorovat.

Zakreslete nalezené defekty do původního obrázku.

In [ ]:
# Pomocná metoda pro vizualizaci
def vizualize(image, image_binary):
    _, _, contours = find_contours(image_binary, 50, fill=False)

    result = to_3_channels(image)
    cv2.drawContours(result, contours, -1, (0, 0, 255), 2)
    return result

In [ ]:
defects_bin = ...(cardboard_filt_final, 50) ### vhodná metoda segmentace

# Vizualizace výsledků
defects_viz = vizualize(cardboard_texture_cropped, defects_bin) 
plot_images(defects_bin, defects_viz)